In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from sklearn.metrics import confusion_matrix


In [5]:
def load_data(data_path):
    data=pd.read_csv(data_path)
    X = data.iloc[: ,:-1].to_numpy()
    Y = data['labels'].to_numpy()

    X = X.reshape(X.shape[0], X.shape[1], 1)
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test

def create_compile_model():
    pass

## MODEL 50

In [6]:
x_train, x_test, y_train, y_test = load_data("../data/data_proccesed_50.csv")

def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.02))
scheduler = LearningRateScheduler(custom_scheduler)

normalizer = layers.Normalization()
normalizer.adapt(x_train)

model_lstm_50 = models.Sequential([
    layers.Input(shape=(x_train.shape[1], x_train.shape[2])), 
    normalizer,
    layers.LSTM(64, activation="tanh"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid"),
])

model_lstm_50.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history_lstm_50=model_lstm_50.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[scheduler])

model_lstm_50.save(r"../model/model_50.h5")


Epoch 1/10
7174/7174 [==============================] - 85s 12ms/step - loss: 0.0353 - accuracy: 0.9914 - val_loss: 0.0145 - val_accuracy: 0.9972 - lr: 0.0010
Epoch 2/10
7174/7174 [==============================] - 83s 12ms/step - loss: 0.0161 - accuracy: 0.9969 - val_loss: 0.0266 - val_accuracy: 0.9943 - lr: 0.0010
Epoch 3/10
7174/7174 [==============================] - 82s 11ms/step - loss: 0.0104 - accuracy: 0.9980 - val_loss: 0.0113 - val_accuracy: 0.9983 - lr: 0.0010
Epoch 4/10
7174/7174 [==============================] - 82s 11ms/step - loss: 0.0086 - accuracy: 0.9984 - val_loss: 0.0043 - val_accuracy: 0.9992 - lr: 0.0010
Epoch 5/10
7174/7174 [==============================] - 83s 12ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.0044 - val_accuracy: 0.9992 - lr: 0.0010
Epoch 6/10
7174/7174 [==============================] - 83s 12ms/step - loss: 0.0051 - accuracy: 0.9991 - val_loss: 0.0023 - val_accuracy: 0.9996 - lr: 0.0010
Epoch 7/10
7174/7174 [========================

c:\Users\Kuba\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## MODEL 100

In [ ]:
x_train, x_test, y_train, y_test = load_data("../data/data_proccesed_100.csv")

model_lstm_100 = models.Sequential([
    layers.Input(shape=(x_train.shape[1], x_train.shape[2])), 
    normalizer,
    layers.LSTM(64, activation="tanh"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid"),
])

model_lstm_100.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history_lstm_100=model_lstm_100.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[scheduler])
model_lstm_100.save(r"../model/model_100.h5")

## MODEL 200

In [ ]:
x_train, x_test, y_train, y_test = load_data("../data/data_proccesed_200.csv")

model_lstm_200 = models.Sequential([
    layers.Input(shape=(x_train.shape[1], x_train.shape[2])), 
    normalizer,
    layers.LSTM(64, activation="tanh"),
    layers.Dropout(0.3),
    layers.Dense(1, activation="sigmoid"),
])

model_lstm_200.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

history_lstm_200=model_lstm_200.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[scheduler])

model_lstm_200.save(r"../model/model_200.h5")

## CONVO

In [ ]:
# def custom_scheduler(epoch, lr):
#     if epoch < 10:
#         return lr
#     else:
#         return float(lr * tf.math.exp(-0.02))
# scheduler = LearningRateScheduler(custom_scheduler)

# normalizer = layers.Normalization()
# normalizer.adapt(x_train)

# model_cnn = models.Sequential([
#     layers.Input(shape=(x_train.shape[1], x_train.shape[2])), 
#     normalizer,
#     layers.Conv1D(64, 1, activation="relu"),
#     # layers.MaxPooling1D(pool_size=2),
#     layers.Dropout(0.3),
#     layers.Flatten(),
#     layers.Dense(1, activation="sigmoid"),
# ])

# model_cnn.compile(
#     optimizer=tf.keras.optimizers.Adam(),
#     loss='binary_crossentropy',
#     metrics=['accuracy'],
# )

# history_cnn=model_cnn.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test), callbacks=[scheduler])